In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [2]:
# 解析高德地图API网址信息
def get_latlng_gaode(address,ak):
    ak= ak
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [3]:
# 获取高德地图API地址信息
def get_receiver_lnglat_gaode(address):
    start_time = time.time()
    re_List_gaode = []
    for b in address:
        print(b)
        re_Dict_gaode = {}
        try:
            temp=get_latlng_gaode(b,ak_gaode)
        except requests.exceptions.ConnectionError: # HTTPError
            re_Dict_gaode["re_location_gaode"] = "0,0"         
        else:
            if (temp['status'] == "0"):
                print('status=0')
                pass
            else:
                if (temp['status'] == "1"):
                    if (temp['count'] == "1"):
                        if('location' in temp['geocodes'][0]):
                            re_Dict_gaode["re_location_gaode"] = temp['geocodes'][0]['location']
                        else:
                            pass
                        if ('formatted_address' in temp['geocodes'][0]):
                            re_Dict_gaode["re_formatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                        else:
                            pass
                        if ('province' in temp['geocodes'][0]):
                            re_Dict_gaode["re_province_gaode"] = temp['geocodes'][0]['province']
                        else:
                            pass
                        if ('country' in temp['geocodes'][0]):
                            re_Dict_gaode["re_country_gaode"] = temp['geocodes'][0]['country']
                        else:
                            pass
                        if ('city' in temp['geocodes'][0]):
                            re_Dict_gaode["re_city_gaode"] = temp['geocodes'][0]['city']
                        else:
                            pass
                        if ('district' in temp['geocodes'][0]):
                            re_Dict_gaode["re_district_gaode"] = temp['geocodes'][0]['district']
                        else: 
                            pass
                        if ('level' in temp['geocodes'][0]):
                            re_Dict_gaode["re_level_gaode"] = temp['geocodes'][0]['level']
                        else:
                            pass
                        if ('township' in temp['geocodes'][0]):
                            re_Dict_gaode["re_township_gaode"] = temp['geocodes'][0]['township']
                        else:
                            pass
                        if ('adcode' in temp['geocodes'][0]):
                            re_Dict_gaode["re_adcode_gaode"] = temp['geocodes'][0]['adcode']
                        else:
                            pass
                        if ('street' in temp['geocodes'][0]):
                            re_Dict_gaode["re_street_gaode"] = temp['geocodes'][0]['street']
                        else:
                            pass
                        if ('number' in temp['geocodes'][0]):
                            re_Dict_gaode["re_number_gaode"] = temp['geocodes'][0]['number']
                        else:
                            pass
                    else:
                        if (temp['count'] == "0"):
                            print("count=0")
                            pass
                else:
                    pass
        re_List_gaode.append(re_Dict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("高德总用时：",end_time-start_time)
    return re_List_gaode

In [4]:
def parse_gaode(df,address):
    # 解析高德
    list_gaode = get_receiver_lnglat_gaode(address)
    # df化(高德)
    df_gaode = pd.DataFrame(list_gaode)
    data_gaode = pd.concat([df,df_gaode],axis=1)
    print("data_gaode:" + str(data_gaode.shape))
    return data_gaode

In [5]:
# ak码
ak_gaode = 'c71d9eda293d20db64955275557d92d4'

In [6]:
# 导入数据
# date = "190416"
# fileName = date + "info_New.xls"
# pinyin = "JDXX"
# inputpath = "/root/lnglat_gaode/"
data = pd.read_excel( r"D:\work\logistica\儋州龙岩关联190426.xlsx")

In [7]:
# 导出数据路径
outpath_gaode = "D:\\work\\logistica\\"
# name_UTF8_gaode = date + "Info_lnglat_" + pinyin + "_gaode.csv"

In [8]:
data.columns

Index(['订单编号', '原始订单', '订单类型', '订单状态', '订单子状态', '取消原因', '订单挂起原因', '安装失败原因',
       '订单来源', '结算状态', '支付平台', '机价款', '店铺', 'AB类型', '拆单类型', '制式类型', '订单金额',
       '支付方式', '支付流水号', '收取卡密号码', '订单生成时间', '支付完成时间', '交易完成时间', '发货时间',
       '物流单回填时间', '用户名', 'QQ号', '入网用户姓名', '入网身份证号', '入网手机号', '联系人电话号码',
       'ICCID', '联系住址', '收货人姓名', '收货地址', '收货人电话号码', '收货人邮箱', '邮编',
       '所在省 / 市 / 县', '入网人所在省/市/县', '号码归属地', '配送方式', '营业厅地址', '配送时间', '优惠劵',
       '优惠券编码', 'F码', 'F码名称', '产品推荐人', 'CPS推荐人', '订单备注', '物流单号', '物流签收时间',
       '承运商', '是否上传签收证明项', '预存款发票类别', '物品费发票抬头', '物品费抬头名称', '发票内容', '企业税号',
       '用户留言', '大小卡类型', '写卡渠道', '卡号', '串号', '销售品编号', '销售品名称', '销售品类型', '销售品数量',
       '销售品价格', '合作商', '实际分账金额', '套餐', '主号码', '副号码', '现金预存款', '可选包', '合约补贴',
       '其他', '靓号低消', '靓号等级', '靓号预存款', '营业厅送货方式', '是否线下模式', '是否线下转线上',
       '线下转线上原因', '营业厅送货iccid', '身份证图片1', '身份证图片2', '身份证图片3', '身份证图片4',
       '实名信息复核中时间', '销售品文本步骤名称', '用户选择的步骤内容', '京东不可达原因', '激活地', '所在省 ', '所在市 ',
       '所在县'],
      dtype='ob

In [11]:
data["address_all"]= data["所在省 "] +"省"+ data["所在市 "] + data["收货地址"]

In [12]:
data["address_all"][0:3]

0                  广东省东莞市石马村蛇岭b区7栋
1            湖南省长沙市湖南长沙市岳麓区梅溪湖金茂酒店
2    吉林省四平市叶赫镇步头三街98号住邦20001单元412室
Name: address_all, dtype: object

In [ ]:
# # 选取有用字段
# df = data[['订单编号','收货人地址','所在省','所在市','所在县']]
# dict_province = {'焦作市':'河南','河南省':'河南','河北省':'河北','贵州省':'贵州','湖南省':'湖南','广东省':'广东','江苏省':'江苏','山西省':'山西',
#     '吉林省':'吉林','山东省':'山东','广西壮族自治区':'广西','湖北省':'湖北','陕西省':'陕西','浙江省':'浙江','福建省':'福建',
#     '内蒙古自治区':'内蒙古','云南省':'云南','辽宁省':'辽宁','江西省':'江西','四川省':'四川','黑龙江省':'黑龙江','安徽省':'安徽',
#     '甘肃省':'甘肃','西藏自治区':'西藏','青海省':'青海','宁夏回族自治区':'宁夏',
#     '海南省':'海南','六安市':'安徽','东莞市':'广东','新疆维吾尔自治区':'新疆'}
# df = df.replace(dict_province)

In [ ]:
# 文件分组并检查
print("data:" + str(data.shape))
print("df:" + str(df.shape))

In [13]:
# 解析高德
data_gaode = parse_gaode(data,data["address_all"])
data_gaode['re_location_gaode'] = data_gaode['re_location_gaode'].map(lambda x:str(x))

广东省东莞市石马村蛇岭b区7栋
湖南省长沙市湖南长沙市岳麓区梅溪湖金茂酒店
吉林省四平市叶赫镇步头三街98号住邦20001单元412室
云南省大理白族自治州右所商贸街
吉林省四平市吉林四平市双辽市双辽市老粮食局家属楼四单元602房
广东省阳江市广东阳江市阳西县阳西县儒洞镇昌利新城12栋1单元
广东省阳江市阳西县儒洞镇美碟婚纱
四川省成都市四川成都市新津县花源镇金秋乐园
河南省焦作市河南焦作市解放区和平中街市场布艺区15号老秦（布艺）
河南省濮阳市河南濮阳市华龙区龙华黄河路谢东北门对过金星幼儿园收
河南省濮阳市河南濮阳市市区开州中路168信金花园3单元A1803
湖南省长沙市湖南长沙市岳麓区大坝咀三期9栋
湖南省长沙市梅溪湖国际新城大坝咀三期9栋
四川省成都市四川成都市武侯区环球中心乐天百货超市后门
陕西省渭南市站南街道朝阳大街渭南师范新校区
湖北省武汉市辽宁省沈阳市
浙江省湖州市浙江湖州市南浔区双林人民医院芭堤雅6号楼2801收
四川省自贡市四川自贡市贡井区贡雷路交叉口北150米全兴收
四川省自贡市四川自贡市贡井区金鱼路与207省道交叉口西150米中国福利彩票
吉林省长春市辽宁省沈阳市
吉林省长春市辽宁省沈阳市
四川省成都市四川成都市金堂县白果镇白回路010乡附近英乐开智
湖北省武汉市湖北武汉市蔡甸区宜黄华泰驾校(树藩大街449附3店)
浙江省台州市浙江台州市路桥区金清镇塘上村
北京省北京市北清路白庙村中街东升公寓1单元1303室
北京省北京市早立庄环路村民委员会302室
河南省安阳市河南省安阳市汤阴县古贤镇东隆化村
四川省乐山市紫云后街62号永顺玻璃
四川省自贡市贡雷路交叉口北150米熙贡府6栋501室
四川省自贡市盐马路交叉口西150米中国福利彩票
四川省自贡市盐马路交叉口西150米中国福利彩票
四川省自贡市杨家坝社区4号楼4单元1806室
四川省乐山市婺嫣街65号潮流前线
四川省自贡市长征大道117号附近龙城曦景20号楼2204
四川省自贡市贡井街院子坝路55号捷仕健身
四川省乐山市紫云街草堂寺110号虫洞台球
四川省自贡市金鱼路151号店福洋中店
北京省北京市辽宁省沈阳市
北京省北京市辽宁省沈阳市
四川省自贡市荣升苑(自犍路南50米)4#8-902室
四川省乐山市滨河路92号酱出名门
四川省自贡市胜利巷14号家利烧饼
河北省承德市河北省承德市宽城满族自治区华

In [14]:
# 合表
split_gaode = pd.DataFrame((x.split(',') for x in data_gaode['re_location_gaode']), index=data_gaode.index, columns=['re_siteLng_gaode','re_siteLat_gaode'])
data_gaode = pd.merge(data_gaode,split_gaode,left_index=True,right_index=True)

In [15]:
# 检查合并后结构
# print("data:" + str(data.shape))
# print("df:" + str(df.shape))
# print("data_gaode:" + str(data_gaode.shape))

data_gaode.to_excel(outpath_gaode + "190426.xlsx",index=False)